In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os



In [ ]:
# Load single image
image_files = sorted([f for f in os.listdir("Images") if f.lower().endswith(".png")])
img_name = image_files[0]

img = cv2.imread(f"Images/{img_name}", cv2.IMREAD_GRAYSCALE)

plt.figure(figsize=(6,6))
plt.imshow(img, cmap="gray")
plt.title("Original TEM Image")
plt.axis("off")
plt.show()


In [ ]:
blur = cv2.GaussianBlur(img, (5,5), 0)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
enhanced = clahe.apply(blur)

plt.figure(figsize=(6,6))
plt.imshow(enhanced, cmap="gray")
plt.title("After Gaussian Blur + CLAHE")
plt.axis("off")
plt.show()


In [ ]:
_, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

plt.figure(figsize=(6,6))
plt.imshow(binary, cmap="gray")
plt.title("After Otsu Thresholding")
plt.axis("off")
plt.show()


In [ ]:
opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
plt.figure(figsize=(6,6))
plt.imshow(opened, cmap="gray")
plt.title("After Morphological opening")
plt.axis("off")
plt.show()

In [ ]:

closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))

plt.figure(figsize=(6,6))
plt.imshow(closed, cmap="gray")
plt.title("After Morphology (Open + Close)")
plt.axis("off")
plt.show()


In [ ]:
def remove_border_near_particles(mask, border_width=5):
    """
    Remove particles that are too close to the image borders.
    """
    import cv2
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)
    cleaned_mask = np.zeros_like(mask)
    h, w = mask.shape

    for label in range(1, num_labels):
        x, y, width, height, area = stats[label]

        # Remove particle if bounding box touches border region
        if x <= border_width or y <= border_width or (x + width) >= (w - border_width) or (y + height) >= (h - border_width):
            continue

        cleaned_mask[labels == label] = 255

    return cleaned_mask


In [ ]:
# Distance transform
dist_transform = cv2.distanceTransform(closed, cv2.DIST_L2, 5)

# Sure foreground
_, sure_fg = cv2.threshold(dist_transform, 0.4 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)

# Sure background and unknown
sure_bg = cv2.dilate(closed, np.ones((3,3), np.uint8), iterations=1)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling
num_markers, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Watershed
img_color = cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR)
markers = cv2.watershed(img_color, markers)

# Build split mask
split_mask = np.zeros_like(closed)
split_mask[markers > 1] = 255

plt.figure(figsize=(6,6))
plt.imshow(split_mask, cmap="gray")
plt.title("After Watershed Splitting")
plt.axis("off")
plt.show()


In [ ]:
# Remove particles near image borders
final_mask = remove_border_near_particles(split_mask, border_width=5)

# visualize
import matplotlib.pyplot as plt
plt.figure(figsize=(6,6))
plt.imshow(final_mask, cmap='gray')
plt.title("Final Mask (Border-near particles removed)")
plt.axis('off')
plt.show()



## PIPELINE FOR ALL IMAGES


In [ ]:
#Remove particles near image borders

def remove_border_near_particles(mask, border_width=5):
    """
    Remove particles that are too close to the image borders.
    """
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)
    cleaned_mask = np.zeros_like(mask)
    h, w = mask.shape

    for label in range(1, num_labels):
        x, y, width, height, area = stats[label]

        if x <= border_width or y <= border_width or (x + width) >= (w - border_width) or (y + height) >= (h - border_width):
            continue

        cleaned_mask[labels == label] = 255

    return cleaned_mask


In [ ]:
# Folders for pipeline
image_folder = "Images"   #Change string for new dataset
mask_folder = "Mask2"     
predicted_folder = "predictedmasks"
os.makedirs(predicted_folder, exist_ok=True)

# Sorted list of images
image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(".png")])

print(f"Found {len(image_files)} images.")


In [ ]:
def segment_nanoparticles(img):
    """
    Classical image processing pipeline for TEM nanoparticles
    """
    #Preprocessing
    blur = cv2.GaussianBlur(img, (5,5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(blur)

    # Binary Thresholding(Otsu)
    _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    #Morphological operations
    opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))

    #Watershed splitting
    dist_transform = cv2.distanceTransform(closed, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.4*dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    sure_bg = cv2.dilate(closed, np.ones((3,3), np.uint8), iterations=1)
    unknown = cv2.subtract(sure_bg, sure_fg)

    num_markers, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    img_color = cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(img_color, markers)

    split_mask = np.zeros_like(closed)
    split_mask[markers > 1] = 255

    #Remove border-near particles
    final_mask = remove_border_near_particles(split_mask, border_width=5)

    return final_mask


In [ ]:
for img_name in image_files:
    # Load image
    img = cv2.imread(os.path.join(image_folder, img_name), cv2.IMREAD_GRAYSCALE)

    # Segment
    pred_mask = segment_nanoparticles(img)

    # Save predicted mask
    save_path = os.path.join(predicted_folder, img_name)
    cv2.imwrite(save_path, pred_mask)

    print(f"Processed: {img_name}")


In [ ]:
for img_name in image_files:
    # Load images
    img = cv2.imread(os.path.join(image_folder, img_name), cv2.IMREAD_GRAYSCALE)
    pred_mask = cv2.imread(os.path.join(predicted_folder, img_name), cv2.IMREAD_GRAYSCALE)
    gt_mask = cv2.imread(os.path.join(mask_folder, img_name), cv2.IMREAD_GRAYSCALE)

    # ----------------------------
                                                        # Create overlay
    
    overlay = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

                                                # True positives (overlap) → white
    overlap = (pred_mask > 0) & (gt_mask > 0)
    overlay[overlap] = [255, 255, 255]

                                                # False positives → orange (predicted only)
    false_pos = (pred_mask > 0) & (gt_mask == 0)
    overlay[false_pos] = [0, 165, 255]  # BGR

                                              # False negatives → purple (ground truth only)
    false_neg = (pred_mask == 0) & (gt_mask > 0)
    overlay[false_neg] = [128, 0, 128]  # BGR

    
    # Plot
    
    plt.figure(figsize=(18,5))

    # Original image
    plt.subplot(1,4,1)
    plt.imshow(img, cmap='gray')
    plt.title("Original Image")
    plt.axis('off')

    # Predicted mask
    plt.subplot(1,4,2)
    plt.imshow(pred_mask, cmap='gray')
    plt.title("Predicted Mask")
    plt.axis('off')

    # Ground truth mask
    plt.subplot(1,4,3)
    plt.imshow(gt_mask, cmap='gray')
    plt.title("Ground Truth (Mask2)")
    plt.axis('off')

    # Overlay
    plt.subplot(1,4,4)
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title("Overlay: Pred vs GT")
    plt.axis('off')

    plt.suptitle(img_name)
    plt.show()




In [ ]:

from scipy.spatial.distance import directed_hausdorff

                                                        # Metric functions

def dice_score(pred_mask, gt_mask):
    pred_bin = (pred_mask > 0).astype(np.uint8)
    gt_bin = (gt_mask > 0).astype(np.uint8)
    intersection = np.sum(pred_bin * gt_bin)
    size_sum = np.sum(pred_bin) + np.sum(gt_bin)
    return 1.0 if size_sum == 0 else 2 * intersection / size_sum

def iou_score(pred_mask, gt_mask):
    pred_bin = (pred_mask > 0).astype(np.uint8)
    gt_bin = (gt_mask > 0).astype(np.uint8)
    intersection = np.sum(pred_bin * gt_bin)
    union = np.sum((pred_bin + gt_bin) > 0)
    return 1.0 if union == 0 else intersection / union

def hausdorff_distance(pred_mask, gt_mask):
    pred_pts = np.column_stack(np.where(pred_mask > 0))
    gt_pts = np.column_stack(np.where(gt_mask > 0))
    if len(pred_pts) == 0 or len(gt_pts) == 0:
        return np.nan
    d1 = directed_hausdorff(pred_pts, gt_pts)[0]
    d2 = directed_hausdorff(gt_pts, pred_pts)[0]
    return max(d1, d2)

def count_particles(mask):
    num_labels, _ = cv2.connectedComponents(mask)
    return num_labels - 1


                                                          # Full evaluation loop

dice_scores = []
iou_scores = []
hausdorff_scores = []
pred_counts = []
gt_counts = []

for img_name in image_files:
    pred_mask = cv2.imread(os.path.join(predicted_folder, img_name), cv2.IMREAD_GRAYSCALE)
    gt_mask = cv2.imread(os.path.join(mask_folder, img_name), cv2.IMREAD_GRAYSCALE)

    dice = dice_score(pred_mask, gt_mask)
    iou = iou_score(pred_mask, gt_mask)
    hd = hausdorff_distance(pred_mask, gt_mask)
    pred_n = count_particles(pred_mask)
    gt_n = count_particles(gt_mask)

    dice_scores.append(dice)
    iou_scores.append(iou)
    hausdorff_scores.append(hd)
    pred_counts.append(pred_n)
    gt_counts.append(gt_n)

    print(f"{img_name}: "
          f"DICE={dice:.3f}, "
          f"IoU={iou:.3f}, "
          f"Hausdorff={hd:.2f}, "
          f"Pred={pred_n}, "
          f"GT={gt_n}")

                                                           # Dataset summary
print("\n========= DATASET SUMMARY =========")
print(f"Mean DICE:        {np.nanmean(dice_scores):.4f}")
print(f"Mean IoU:         {np.nanmean(iou_scores):.4f}")
print(f"Mean Hausdorff:   {np.nanmean(hausdorff_scores):.2f}")
print(f"Mean Prediction Count:  {np.mean(pred_counts):.1f}")
print(f"Mean Ground Truth Count:    {np.mean(gt_counts):.1f}")
print(f"Mean Count Error: {np.mean(np.abs(np.array(pred_counts)-np.array(gt_counts))):.2f}")
